In [15]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import Axes
from matplotlib.animation import FuncAnimation
from matplotlib.colors import Normalize

from hysteresis import vb, vw, equilibrium
from helper import _T_p, celsius, solve_varying_L_quasi_static, kelvin
from plotting import add_arrow

# 2 Black & White Daisyworld


## 2.2 State-Space Picture


In [ ]:
L = 1.0
Aw, Ab = np.mgrid[0:1:100j, 0:1:100j]
velb = vb(Ab, Aw, L)
velw = vw(Ab, Aw, L)

# Impose Region Constraint (Ab + Aw >= 1)
mask = np.zeros(velw.shape, dtype=bool)
for i in range(len(velw)):
    velw[i, len(velw) - i :] = np.nan
    velw = np.ma.array(velw, mask=mask)

for i in range(len(velb)):
    velb[i, len(velb) - i :] = np.nan
    velb = np.ma.array(velb, mask=mask)

fig = plt.figure()
ax = fig.add_subplot()
ax.set_title(f"$L$ = {L}")
ax.set_xlabel("$A_b$")
ax.set_ylabel("$A_w$")
ax.set_aspect("equal")
ax.set_xlim(-0.05, 1.0)
ax.set_ylim(-0.05, 1.0)
normalize = Normalize(vmin=-0.2, vmax=0.2)
im = ax.contourf(Ab, Aw, velw, cmap="bwr", norm=normalize)
fig.colorbar(mappable=im)
fig.savefig(
    "presentation_media/state_space_dA_w.svg",
    transparent=True,
    bbox_inches="tight",
    pad_inches=0.0,
)
fig.show()


fig = plt.figure()
ax = fig.add_subplot()
ax.set_title(f"$L$ = {L}")
ax.set_xlabel("$A_b$")
ax.set_ylabel("$A_w$")
ax.set_aspect("equal")
ax.set_xlim(-0.05, 1.0)
ax.set_ylim(-0.05, 1.0)
normalize = Normalize(vmin=-0.1, vmax=0.1)
im = ax.contourf(Ab, Aw, velb, cmap="bwr", norm=normalize)
fig.colorbar(mappable=im)
fig.savefig(
    "presentation_media/state_space_dA_b.svg",
    transparent=True,
    bbox_inches="tight",
    pad_inches=0.0,
)
fig.show()

In [ ]:
def plot_state_space(ax: Axes, L):
    Aw, Ab = np.mgrid[0:1:100j, 0:1:100j]
    velb = vb(Ab, Aw, L)
    velw = vw(Ab, Aw, L)

    # Impose Region Constraint (Ab + Aw >= 1)
    mask = np.zeros(velw.shape, dtype=bool)
    for i in range(len(velw)):
        velw[i, len(velw) - i :] = np.nan
        velw = np.ma.array(velw, mask=mask)

    # Equilibrium points
    stable_points, unstable_points = equilibrium(L=L)

    ax.plot(*stable_points.T, "b^")
    ax.plot(*unstable_points.T, "rv")
    ax.streamplot(Ab, Aw, velb, velw, color="k", density=1.5, linewidth=0.75)


for L in [0.66, 0.8, 1.0, 1.15, 1.3, 1.45]:
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.set_title(f"$L$ = {L}")
    ax.set_xlabel("$A_b$")
    ax.set_ylabel("$A_w$")
    ax.set_aspect("equal")
    ax.set_xlim(-0.05, 1.0)
    ax.set_ylim(-0.05, 1.0)

    plot_state_space(ax, L)
    fig.savefig(f"presentation_media/state_space_L_{L}.svg", transparent=True)
    fig.show()

In [ ]:
slide_bg = np.array([255 / 255, 242 / 255, 204 / 255, 1])

fig = plt.figure()
fig.patch.set_facecolor(slide_bg)
ax = fig.add_subplot()
ax.set_facecolor(slide_bg)
ax.set_xlabel("$A_b$")
ax.set_ylabel("$A_w$")
ax.set_aspect("equal")
ax.set_xlim(-0.05, 1.0)
ax.set_ylim(-0.05, 1.0)


# Animation function
def animate(L):
    print(L)
    ax.clear()
    ax.set_title(f"$L$ = {np.round(L, 2)}")
    plot_state_space(ax, L)


L_values = np.linspace(0.8, 1.8, 100)  # Adjust range and step as needed
ani = FuncAnimation(fig, animate, frames=L_values, interval=100)

# Save or show the animation
ani.save("state_space_animation.gif", writer="imagemagick")  # Save as a GIF
# plt.show()  # Uncomment to just display the animation

### Bifurcation Plot


In [7]:
%matplotlib inline
L_begin = 0.4
L_end = 1.8
dL = 0.01

stable_point_set, unstable_point_set = [], []
Ls = np.arange(L_begin, L_end, dL)

for L in Ls:
    stable_points, unstable_points = equilibrium(L=L)
    stable_point_set.extend(np.hstack([stable_points, np.repeat(L, len(stable_points))[:, np.newaxis]]))
    unstable_point_set.extend(np.hstack([unstable_points, np.repeat(L, len(unstable_points))[:, np.newaxis]]))

In [ ]:
%matplotlib inline

stable_point_set = np.array(stable_point_set)
unstable_point_set = np.array(unstable_point_set)

# Plotting
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Plot L on x-axis, A_b on y-axis
def transform_for_plot(points):
    return [points.T[i] for i in [2,0,1]]

ax.plot(
    *transform_for_plot(stable_point_set),
    "b^",
    markersize=7,
)
ax.plot(
    *transform_for_plot(unstable_point_set),
    "rv",
    markersize=7,
)

# Formatting
ax.set_xlim(L_begin, L_end)
ax.set_ylim(0, 1)
ax.set_zlim(0, 1)

# ax.set_xlabel("$L$")
# ax.set_ylabel("$A_b$")
# ax.set_zlabel("$A_w$")

ax.view_init(azim=-140, elev=30)  # adjust the view angle
plt.savefig("presentation_media/stable_unstable_points.svg", transparent=True, bbox_inches="tight", pad_inches=0.0)
plt.show()

In [ ]:
%matplotlib inline
# Original axes: A_b on x, A_w on y, L on z
e0_0 = stable_point_set[(stable_point_set[:, 2] < 0.8) & (stable_point_set[:, 0] < 0.01)]
e0_1 = stable_point_set[(stable_point_set[:, 2] > 0.8) & (stable_point_set[:, 1] < 0.01)]
e3 = unstable_point_set[(0.6 < unstable_point_set[:, 2]) & (unstable_point_set[:, 2] < 0.71)]
e4 = unstable_point_set[(unstable_point_set[:, 2] > 0.71) & (unstable_point_set[:, 2] < 1.2) & (unstable_point_set[:, 0] > 0.01)]
e1 = unstable_point_set[(unstable_point_set[:, 2] > 0.71) & (unstable_point_set[:, 2] < 1.25) & (unstable_point_set[:, 0] < 0.01) & (unstable_point_set[:, 1] > 0.01)]
e2 = unstable_point_set[(unstable_point_set[:, 2] > 1.25) & (unstable_point_set[:, 0] < 0.01) & (unstable_point_set[:, 1] > 0.01)]
e5 = stable_point_set[(stable_point_set[:, 0] > 0.001) & (stable_point_set[:, 1] > 0.001)]
s0 = unstable_point_set[(unstable_point_set[:, 0]< 0.001) & (unstable_point_set[:, 1] < 0.001)]
s1 = stable_point_set[(stable_point_set[:, 0] > 0.4) & (stable_point_set[:, 1] < 0.01)]
s2 = stable_point_set[(stable_point_set[:, 1] > 0.4) & (stable_point_set[:, 0] < 0.01)]

transcritical_points = np.array([e5[0], e5[-1], e3[-1], e4[-1], e1[0], e2[0]])
saddle_points = np.array([e3[0], e2[-1]])

# Plotting
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.plot(*transform_for_plot(e0_0), color="k")
ax.plot(*transform_for_plot(e0_1), color="k")
ax.plot(*transform_for_plot(s0), color="k", linestyle="--")
ax.plot(*transform_for_plot(s2), color="C0")
ax.plot(*transform_for_plot(e1), color="C0", linestyle="--")
ax.plot(*transform_for_plot(e2), color="C0", linestyle="--")
ax.plot(*transform_for_plot(s1), color="C1")
ax.plot(*transform_for_plot(e3), color="C1", linestyle="--")
ax.plot(*transform_for_plot(e4), color="C1", linestyle="--")
ax.plot(*transform_for_plot(e5), color="C2")


# Formatting
ax.set_xlim(L_begin, L_end)
ax.set_ylim(0, 1)
ax.set_zlim(0, 1)

# ax.set_xlabel("$L$")
# ax.set_ylabel("$A_b$")
# ax.set_zlabel("$A_w$")

ax.view_init(azim=-140, elev=30)
plt.savefig("presentation_media/state_transitions.svg", transparent=True, bbox_inches="tight", pad_inches=0.0)

ax.plot(*transform_for_plot(saddle_points), "C3o", markersize=5)
ax.plot(*transform_for_plot(transcritical_points), "C4o", markersize=5)
plt.savefig("presentation_media/state_transitions_with_points.svg", transparent=True, bbox_inches="tight", pad_inches=0.0)
plt.show()

In [24]:
Ls_1 = np.linspace(0.4, 1.8, 500)
A_w, A_b = 0.5, 0.5
T_ps_1, A_ws_1, A_bs_1 = solve_varying_L_quasi_static(
    Ls_1, A_w0=A_w, A_b0=A_b, D_f=0.003265, T_opt=kelvin(22.5)
)

Ls_2 = np.linspace(1.8, 0.4, 500)
A_w, A_b = 0.5, 0.5
T_ps_2, A_ws_2, A_bs_2 = solve_varying_L_quasi_static(
    Ls_2, A_w0=A_w, A_b0=A_b, D_f=0.003265, T_opt=kelvin(22.5)
)

In [ ]:
%matplotlib inline

def points_to_temperature(points):
    return celsius(_T_p(A_w=points[:, 1], A_b=points[:, 0], L=points[:, 2]))

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(e0_0[:, 2], points_to_temperature(e0_0), color="k")
ax.plot(e0_1[:, 2], points_to_temperature(e0_1), color="k")
ax.plot(s0[:, 2], points_to_temperature(s0), color="k", linestyle="--")
ax.plot(s2[:, 2], points_to_temperature(s2), color="C0")
ax.plot(e1[:, 2], points_to_temperature(e1), color="C0", linestyle="--")
ax.plot(e2[:, 2], points_to_temperature(e2), color="C0", linestyle="--")
ax.plot(s1[:, 2], points_to_temperature(s1), color="C1")
ax.plot(e3[:, 2], points_to_temperature(e3), color="C1", linestyle="--")
ax.plot(e4[:, 2], points_to_temperature(e4), color="C1", linestyle="--")
ax.plot(e5[:, 2], points_to_temperature(e5), color="C2")


ax.plot(saddle_points[:, 2], points_to_temperature(saddle_points), "C3o", markersize=5)
ax.plot(transcritical_points[:, 2], points_to_temperature(transcritical_points), "C4o", markersize=5)

plt.xlabel("$L$")
plt.ylabel("Temperature ($\\degree C$)")
plt.xlim(0.4, 1.8)
plt.ylim(-40, 80)
plt.savefig("presentation_media/state_space_temperature.svg", transparent=True)

mask1 = (0.6 < Ls_1) & (Ls_1 < 0.725)
(line1,) = ax.plot(Ls_1[mask1], celsius(np.array(T_ps_1))[mask1], color="r")
add_arrow(line1, at_pos=0.5)
plt.savefig("presentation_media/state_space_temperature_path_0.svg", transparent=True)

mask1 = (0.6 < Ls_1) & (Ls_1 < 1.0)
(line1,) = ax.plot(Ls_1[mask1], celsius(np.array(T_ps_1))[mask1], color="r")
add_arrow(line1, at_pos=0.5)
plt.savefig("presentation_media/state_space_temperature_path_1.svg", transparent=True)

mask1 = (0.6 < Ls_1) & (Ls_1 < 1.4)
(line1,) = ax.plot(Ls_1[mask1], celsius(np.array(T_ps_1))[mask1], color="r")
add_arrow(line1, at_pos=0.9)
plt.savefig("presentation_media/state_space_temperature_path_2.svg", transparent=True)

mask1 = (0.6 < Ls_1) & (Ls_1 < 1.7)
(line1,) = ax.plot(Ls_1[mask1], celsius(np.array(T_ps_1))[mask1], color="r")
add_arrow(line1, at_pos=0.975)
plt.savefig("presentation_media/state_space_temperature_path_3.svg", transparent=True)

mask1 = (0.55 < Ls_2) & (Ls_2 < 1.7)
(line1,) = ax.plot(Ls_2[mask1], celsius(np.array(T_ps_2))[mask1], color="b")
add_arrow(line1, at_pos=0.8)
add_arrow(line1, at_pos=0.6)
add_arrow(line1, at_pos=0.375)
add_arrow(line1, at_pos=0.1)
add_arrow(line1, at_pos=0.025)
plt.savefig("presentation_media/state_space_temperature_path_4.svg", transparent=True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()

plt.xlabel("$L$")
plt.ylabel("Temperature ($\\degree C$)")
plt.xlim(0.4, 1.8)
plt.ylim(-40, 80)

mask1 = (0.6 < Ls_1) & (Ls_1 < 0.725)
(line1,) = ax.plot(Ls_1[mask1], celsius(np.array(T_ps_1))[mask1], color="r")
add_arrow(line1, at_pos=0.5)

mask1 = (0.6 < Ls_1) & (Ls_1 < 1.0)
(line1,) = ax.plot(Ls_1[mask1], celsius(np.array(T_ps_1))[mask1], color="r")
add_arrow(line1, at_pos=0.5)

mask1 = (0.6 < Ls_1) & (Ls_1 < 1.4)
(line1,) = ax.plot(Ls_1[mask1], celsius(np.array(T_ps_1))[mask1], color="r")
add_arrow(line1, at_pos=0.9)

mask1 = (0.6 < Ls_1) & (Ls_1 < 1.7)
(line1,) = ax.plot(Ls_1[mask1], celsius(np.array(T_ps_1))[mask1], color="r")
add_arrow(line1, at_pos=0.975)

mask1 = (0.55 < Ls_2) & (Ls_2 < 1.7)
(line1,) = ax.plot(Ls_2[mask1], celsius(np.array(T_ps_2))[mask1], color="b")
add_arrow(line1, at_pos=0.8)
add_arrow(line1, at_pos=0.6)
add_arrow(line1, at_pos=0.375)
add_arrow(line1, at_pos=0.1)
add_arrow(line1, at_pos=0.025)
plt.savefig(
    "presentation_media/state_space_temperature_path_only.svg", transparent=True
)